In [26]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"SQLite version: {sqlite3.version}")
        
        # Let's check the actual structure of your tables
        check_table_structure(conn)
        
    except Error as e:
        print(e)
 
    return conn

def check_table_structure(conn):
    """Check the structure of existing tables to help debug"""
    cur = conn.cursor()
    
    # Check if Bookings table exists and its structure
    cur.execute("PRAGMA table_info(Bookings)")
    bookings_columns = cur.fetchall()
    print("\nBookings table columns:")
    for col in bookings_columns:
        print(col)
    
    # Check if Facilities table exists and its structure
    cur.execute("PRAGMA table_info(Facilities)")
    facilities_columns = cur.fetchall()
    print("\nFacilities table columns:")
    for col in facilities_columns:
        print(col)

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    # --- THIS IS THE SQL QUERY FOR Q10 ---
    # Using initialcost 
    query1 = """
        SELECT 
            f.name,
            SUM(CASE 
                WHEN b.memid = 0 THEN b.slots * f.initialcost  -- Changed from guestcost
                ELSE b.slots * f.initialcost                   -- Changed from membercost
            END) AS total_revenue
        FROM 
            Bookings b
        JOIN 
            Facilities f ON b.facid = f.facid
        GROUP BY 
            f.name
        HAVING 
            total_revenue < 1000
        ORDER BY 
            total_revenue;
        """
    
    try:
        cur.execute(query1)
        rows = cur.fetchall()
        print("\n--- Q10 Results (Facilities with Total Revenue < 1000) ---")
        for row in rows:
            print(row)
            
    except Error as e:
        print(f"Query error: {e}")
        print("\nTrying to determine the correct column names...")
        
        # If the query fails, let's try to find the correct column names
        cur.execute("SELECT * FROM Bookings LIMIT 1")
        bookings_cols = [description[0] for description in cur.description]
        print(f"Bookings columns: {bookings_cols}")
        
        cur.execute("SELECT * FROM Facilities LIMIT 1")
        facilities_cols = [description[0] for description in cur.description]
        print(f"Facilities columns: {facilities_cols}")


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    if conn: 
        with conn: 
            select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

SQLite version: 2.6.0

Bookings table columns:
(0, 'bookid', 'INTEGER', 0, None, 1)
(1, 'facid', 'INTEGER', 0, None, 0)
(2, 'memid', 'INTEGER', 0, None, 0)
(3, 'slots', 'INTEGER', 0, None, 0)

Facilities table columns:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 1, None, 0)
(2, 'location', 'TEXT', 0, None, 0)
(3, 'capacity', 'INTEGER', 0, None, 0)
Query error: no such column: f.initialcost

Trying to determine the correct column names...
Bookings columns: ['bookid', 'facid', 'memid', 'slots']
Facilities columns: ['id', 'name', 'location', 'capacity']


In [33]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn


def create_table(conn):
    """
    Create the Members table if it doesn't exist
    :param conn: Connection object
    :return:
    """
    try:
        cur = conn.cursor()
        # Create the Members table with necessary columns
        cur.execute('''
            CREATE TABLE IF NOT EXISTS Members (
                memid INTEGER PRIMARY KEY,
                firstname TEXT NOT NULL,
                surname TEXT NOT NULL,
                recommendedby INTEGER,
                FOREIGN KEY (recommendedby) REFERENCES Members(memid)
            )
        ''')
        
        # Insert some sample data
        sample_data = [
            (1, 'John', 'Doe', None),
            (2, 'Jane', 'Smith', 1),
            (3, 'Bob', 'Johnson', 1),
            (4, 'Alice', 'Williams', 2)
        ]
        
        cur.executemany('INSERT OR IGNORE INTO Members VALUES (?,?,?,?)', sample_data)
        conn.commit()
    except Error as e:
        print(e)

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    #--- THIS IS THE SQL QUERY FOR Q11 ---
    query1 = """
        SELECT 
            (m1.firstname || ' ' || m1.surname) AS member_name,
            (m2.firstname || ' ' || m2.surname) AS recommender_name
        FROM 
            Members m1
        JOIN 
            Members m2 ON m1.recommendedby = m2.memid
        ORDER BY 
            m1.surname, m1.firstname;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        # First create the table and add sample data
        create_table(conn)
        
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Bob Johnson', 'John Doe')
('Jane Smith', 'John Doe')
('Alice Williams', 'Jane Smith')


In [28]:

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        # This print statement just confirms the library version
        print(f"SQLite version: {sqlite3.version}")
        
        # Let's check the actual structure of your tables
        check_table_structure(conn)
        
    except Error as e:
        print(e)
 
    return conn

def check_table_structure(conn):
    """Check the structure of existing tables to help debug"""
    cur = conn.cursor()
    
    # Check if Bookings table exists and its structure
    cur.execute("PRAGMA table_info(Bookings)")
    bookings_columns = cur.fetchall()
    print("\nBookings table columns:")
    for col in bookings_columns:
        print(col)
    
    # Check if Facilities table exists and its structure
    cur.execute("PRAGMA table_info(Facilities)")
    facilities_columns = cur.fetchall()
    print("\nFacilities table columns:")
    for col in facilities_columns:
        print(col)

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    #--- THIS IS THE SQL QUERY FOR Q12 ---
    query1 = """
        SELECT 
            f.name,
            SUM(b.slots) AS total_member_slots
        FROM 
            Bookings b
        JOIN 
            Facilities f ON b.facid = f.facid
        WHERE 
            b.memid != 0
        GROUP BY 
            f.name
        ORDER BY 
            f.name;
        """
    
    try:
        cur.execute(query1)
        rows = cur.fetchall()
        print("\n--- Q12 Results (Facility Usage by Members) ---")
        for row in rows:
            print(row)
    except Error as e:
        print(f"Query error: {e}")
        print("\nTrying to determine the correct column names...")
        
        # If the query fails, let's try to find the correct column names
        cur.execute("SELECT * FROM Bookings LIMIT 1")
        bookings_cols = [description[0] for description in cur.description]
        print(f"Bookings columns: {bookings_cols}")
        
        cur.execute("SELECT * FROM Facilities LIMIT 1")
        facilities_cols = [description[0] for description in cur.description]
        print(f"Facilities columns: {facilities_cols}")


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    if conn: 
        with conn: 
            select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()
 


SQLite version: 2.6.0

Bookings table columns:
(0, 'bookid', 'INTEGER', 0, None, 1)
(1, 'facid', 'INTEGER', 0, None, 0)
(2, 'memid', 'INTEGER', 0, None, 0)
(3, 'slots', 'INTEGER', 0, None, 0)

Facilities table columns:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 1, None, 0)
(2, 'location', 'TEXT', 0, None, 0)
(3, 'capacity', 'INTEGER', 0, None, 0)
Query error: no such column: f.facid

Trying to determine the correct column names...
Bookings columns: ['bookid', 'facid', 'memid', 'slots']
Facilities columns: ['id', 'name', 'location', 'capacity']


In [29]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        # This print statement just confirms the library version
        print(f"SQLite version: {sqlite3.version}")
        
        # Let's check the actual structure of your tables
        check_table_structure(conn)
        
    except Error as e:
        print(e)
 
    return conn

def check_table_structure(conn):
    """Check the structure of existing tables to help debug"""
    cur = conn.cursor()
    
    # Check if Bookings table exists and its structure
    cur.execute("PRAGMA table_info(Bookings)")
    bookings_columns = cur.fetchall()
    print("\nBookings table columns:")
    for col in bookings_columns:
        print(col)
    
    # Check if Facilities table exists and its structure
    cur.execute("PRAGMA table_info(Facilities)")
    facilities_columns = cur.fetchall()
    print("\nFacilities table columns:")
    for col in facilities_columns:
        print(col)

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    # --- THIS IS THE SQL QUERY FOR Q13 ---
    query1 = """
        SELECT 
            f.name,
            STRFTIME('%Y-%m', b.starttime) AS month,
            SUM(b.slots) AS total_member_slots
        FROM 
            Bookings b
        JOIN 
            Facilities f ON b.facid = f.facid
        WHERE 
            b.memid != 0
        GROUP BY 
            f.name, month
        ORDER BY 
            f.name, month;
        """
    
    try:
        cur.execute(query1)
        rows = cur.fetchall()
        print("\n--- Q13 Results (Facility Usage by Members per Month) ---")
        for row in rows:
            print(row)
    except Error as e:
        print(f"Query error: {e}")
        print("\nTrying to determine the correct column names...")
        
        # If the query fails, let's try to find the correct column names
        cur.execute("SELECT * FROM Bookings LIMIT 1")
        bookings_cols = [description[0] for description in cur.description]
        print(f"Bookings columns: {bookings_cols}")
        
        cur.execute("SELECT * FROM Facilities LIMIT 1")
        facilities_cols = [description[0] for description in cur.description]
        print(f"Facilities columns: {facilities_cols}")


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    if conn: 
        with conn: 
            select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

SQLite version: 2.6.0

Bookings table columns:
(0, 'bookid', 'INTEGER', 0, None, 1)
(1, 'facid', 'INTEGER', 0, None, 0)
(2, 'memid', 'INTEGER', 0, None, 0)
(3, 'slots', 'INTEGER', 0, None, 0)

Facilities table columns:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 1, None, 0)
(2, 'location', 'TEXT', 0, None, 0)
(3, 'capacity', 'INTEGER', 0, None, 0)
Query error: no such column: b.starttime

Trying to determine the correct column names...
Bookings columns: ['bookid', 'facid', 'memid', 'slots']
Facilities columns: ['id', 'name', 'location', 'capacity']
